# Import necessary moudules

In [1]:
import numpy as np
import anndata as ad
from typing import Optional
from FeatureSelectionBenchmarks.benchmark.run_benchmark import run_bench

# Define custom methods

In [2]:
def random_select(adata: ad.AnnData, n_selected_genes: int, seed: int):
    rng = np.random.default_rng(seed)
    return rng.choice(adata.var_names, size=n_selected_genes)


def random_clustering(adata: ad.AnnData, img: Optional[np.ndarray], k: int, seed: int):
    rng = np.random.default_rng(seed)
    return rng.integers(low=0, high=k, size=adata.n_obs)

# Three configurations about data, feature selection, and downstream analysis

Before running the benchmark, you first need to prepare three configurations stored in `dict` type: `data_cfg`, `fs_cfg`, and `cl_cfg`.

## 1. data_cfg

Configurations of datasets. It should be a dict in the format `{'data_name': {'property_name': data_property}}`. 
Supported property names are:

- 'adata_path': path to the `h5ad` file.
- 'image_path': path to the `h5ad` file, optional. It will be ignored when `modality='scrna'`.
- 'annot_key' : a key in `adata.obs` that represents annotations (cell types/domains).
- 'to_replace': replace some values in `adata.obs['annot_key']`, optional. It should be a dict in the format
 `{'value': ['to_replace_1', 'to_replace_2']}`, where 'to_replace_x' are values that will be replaced, and
 'value' is value to replace any values matching 'to_replace_x' with.
- 'to_remove' :  remove cells/spots that have some values in `adata.obs['annot_key']`, optional. It should be a
list of values to be removed.
- 'batch': a key in `adata.obs` that represents batches, optional. Only valid when `modality='scrna'`. If it's
specified, the benchmark will use the algorithm in Seurat to combine the features selected in each batch.
- 'shape': Set 'hexagon' for Visium data, and 'square' for ST data when `modality='spatial'`, optional.
Default is 'hexagon'. Currently this parameter is only used in the cluster refinement of `spaGCN`.

## 2. fs_cfg

Configurations of feature selection methods. It should be a dict in the format
`{fs_method: list_of_numbers_of_selected_genes}`. More specifically,

- fs_method: can be either a string that represents a predefined function in this benchmark, or a custom
function. The benchmark will call the function like `custom_fs_function(adata, n_selected_genes, **kwargs)`,
and the return values must be an ndarray that contains features selected by the function. You can write a
wrapper function to work around incompatible parameters/return values.
- list_of_numbers_of_selected_genes: a list of numbers of genes needed to be selected. If the function
internally determines the number of selected genes (e.g. GeneClust), write the list as `['auto']`.


## 3. cl_cfg

Configurations of downstream cell clustering/domain detection methods. It should be a dict in the format
`{cl_method: list_of_numbers_of_runs}`. More specifically,

- cl_method: can be either a string that represents a predefined function in this benchmark, or a custom
function. The benchmark will call the function like `custom_cl_function(fs_adata, img, **kwargs)`,
and the return values must be an ndarray that contains cluster labels generated by the function. the parameter
'img' is always `None` when `modality='scrna'`. You can write a wrapper function to work around incompatible
parameters/return values.
- list_of_numbers_of_runs: a list of numbers of times that the function will run with different random states.

# Run benchmark on scRNA-seq data

## Evaluate predefined methods by the benchmark

In [3]:
data_cfg = {'PBMC3k': {
        'adata_path': '../tests/data/scrna/pbmc3k_raw.h5ad',
        'annot_key': 'louvain',
        'to_replace': {'Dendritic': ['Dendritic cells']},
        'to_remove': [np.nan, 'Megakaryocytes']
    }}
fs_cfg = {'seurat_v3': [1000, 2000], 'GeneClust-fast': ['auto']}
cl_cfg = {'KMeans': 2}

run_bench(data_cfg, fs_cfg, cl_cfg, ['ARI', 'NMI'], 'scrna', clean_cache=True)

2023-02-10 14:54:18.842 | INFO  | ./cache has been deleted.


───────────────────────────────────────────────────── PBMC3k ──────────────────────────────────────────────────────

2023-02-10 14:54:18.857 | INFO  | Reading adata...
2023-02-10 14:54:18.858 | INFO  | No cache for PBMC3k. Trying to read h5ad from the given path in config...
2023-02-10 14:54:18.924 | INFO  | Found sparse matrix in `adata.X`. Converting to ndarray...


Before QC: 2700 cells and 32738 genes.

2023-02-10 14:54:19.129 | INFO  | Observation names and Variables names are all unique now.
2023-02-10 14:54:19.417 | INFO  | Dropping 10 special genes from 32738 genes...


...

2023-02-10 14:54:21.062 | INFO  | Running seurat_v3 feature selection with 1000 features to be selected.
2023-02-10 14:54:22.175 | INFO  | Feature selection finished.
2023-02-10 14:54:22.178 | INFO  | Selected genes have been cached.
2023-02-10 14:54:22.279 | INFO  | Running KMeans clustering with 7 clusters and random state 0...
2023-02-10 14:54:22.499 | INFO  | KMeans clustering results have been cached.
2023-02-10 14:54:22.518 | INFO  | Running KMeans clustering with 7 clusters and random state 1...
2023-02-10 14:54:22.666 | INFO  | KMeans clustering results have been cached.


───────────────────────────────────── FS method: seurat_v3, Gene Number: 2000 ─────────────────────────────────────

2023-02-10 14:54:22.678 | INFO  | Running seurat_v3 feature selection with 2000 features to be selected.
2023-02-10 14:54:23.901 | INFO  | Feature selection finished.
2023-02-10 14:54:23.905 | INFO  | Selected genes have been cached.
2023-02-10 14:54:24.065 | INFO  | Running KMeans clustering with 7 clusters and random state 0...
2023-02-10 14:54:24.225 | INFO  | KMeans clustering results have been cached.
2023-02-10 14:54:24.235 | INFO  | Running KMeans clustering with 7 clusters and random state 1...
2023-02-10 14:54:24.348 | INFO  | KMeans clustering results have been cached.


────────────────────────────────── FS method: GeneClust-fast, Gene Number: auto ───────────────────────────────────

2023-02-10 14:54:24.360 | INFO  | Running GeneClust-fast feature selection with auto features to be selected.
2023-02-10 14:54:24.682 | INFO  | Performing GeneClust-fast on scRNA-seq data, with 95 workers.
2023-02-10 14:54:24.732 | INFO  | Preprocessing data...
2023-02-10 14:54:25.724 | INFO  | Data preprocessing done.
2023-02-10 14:54:25.725 | INFO  | Clustering genes...
2023-02-10 14:54:26.024 | INFO  | Gene clustering done!
2023-02-10 14:54:26.491 | INFO  | Selected 396 genes.
2023-02-10 14:54:26.492 | INFO  | GeneClust-fast finished.
2023-02-10 14:54:26.569 | INFO  | Feature selection finished.
2023-02-10 14:54:26.573 | INFO  | Selected genes have been cached.
2023-02-10 14:54:26.644 | INFO  | Running KMeans clustering with 7 clusters and random state 0...
2023-02-10 14:54:26.741 | INFO  | KMeans clustering results have been cached.
2023-02-10 14:54:26.751 | INFO  | Running KMeans clustering with 7 clusters and random state 1...
2023-02-10 14:54:26.807 | INFO  | KMeans clustering r

## Evaluate custom methods defined by the user

In [4]:
data_cfg = {'PBMC3k': {
        'adata_path': '../tests/data/scrna/pbmc3k_raw.h5ad',
        'annot_key': 'louvain',
        'to_replace': {'Dendritic': ['Dendritic cells']},
        'to_remove': [np.nan, 'Megakaryocytes']
    }}
fs_cfg = {'seurat_v3': [1000, 2000], random_select: [500]}
cl_cfg = {'KMeans': 2,  random_clustering: 1}

run_bench(data_cfg, fs_cfg, cl_cfg, ['ARI', 'NMI'], 'scrna', clean_cache=True, fs_kwarg={'seed': 123}, cl_kwarg={'k':5, 'seed': 123})

2023-02-10 14:54:26.935 | INFO  | ./cache has been deleted.


───────────────────────────────────────────────────── PBMC3k ──────────────────────────────────────────────────────

2023-02-10 14:54:26.939 | INFO  | Reading adata...
2023-02-10 14:54:26.940 | INFO  | No cache for PBMC3k. Trying to read h5ad from the given path in config...
2023-02-10 14:54:27.006 | INFO  | Found sparse matrix in `adata.X`. Converting to ndarray...


Before QC: 2700 cells and 32738 genes.

2023-02-10 14:54:27.212 | INFO  | Observation names and Variables names are all unique now.
2023-02-10 14:54:27.500 | INFO  | Dropping 10 special genes from 32738 genes...


...

2023-02-10 14:54:29.161 | INFO  | Running seurat_v3 feature selection with 1000 features to be selected.
2023-02-10 14:54:30.212 | INFO  | Feature selection finished.
2023-02-10 14:54:30.215 | INFO  | Selected genes have been cached.
2023-02-10 14:54:30.316 | INFO  | Running KMeans clustering with 7 clusters and random state 0...
2023-02-10 14:54:30.434 | INFO  | KMeans clustering results have been cached.
2023-02-10 14:54:30.444 | INFO  | Running KMeans clustering with 7 clusters and random state 1...
2023-02-10 14:54:30.526 | INFO  | KMeans clustering results have been cached.
2023-02-10 14:54:30.538 | INFO  | random_clustering clustering results have been cached.


───────────────────────────────────── FS method: seurat_v3, Gene Number: 2000 ─────────────────────────────────────

2023-02-10 14:54:30.548 | INFO  | Running seurat_v3 feature selection with 2000 features to be selected.
2023-02-10 14:54:31.766 | INFO  | Feature selection finished.
2023-02-10 14:54:31.769 | INFO  | Selected genes have been cached.
2023-02-10 14:54:31.925 | INFO  | Running KMeans clustering with 7 clusters and random state 0...
2023-02-10 14:54:32.060 | INFO  | KMeans clustering results have been cached.
2023-02-10 14:54:32.070 | INFO  | Running KMeans clustering with 7 clusters and random state 1...
2023-02-10 14:54:32.181 | INFO  | KMeans clustering results have been cached.
2023-02-10 14:54:32.193 | INFO  | random_clustering clustering results have been cached.


─────────────────────────────────── FS method: random_select, Gene Number: 500 ────────────────────────────────────

2023-02-10 14:54:32.205 | INFO  | Selected genes have been cached.
2023-02-10 14:54:32.283 | INFO  | Running KMeans clustering with 7 clusters and random state 0...
2023-02-10 14:54:32.344 | INFO  | KMeans clustering results have been cached.
2023-02-10 14:54:32.354 | INFO  | Running KMeans clustering with 7 clusters and random state 1...
2023-02-10 14:54:32.413 | INFO  | KMeans clustering results have been cached.
2023-02-10 14:54:32.424 | INFO  | random_clustering clustering results have been cached.
2023-02-10 14:54:32.476 | INFO  | records have been saved into './2023-02 14_54_32 scrna.xlsx'.


# Run benchmark on SRT data

## Evaluate predefined methods by the benchmark

In [5]:
data_cfg = {'mouse_brain': {
    'adata_path': '../tests/data/spatial/V1_Adult_Mouse_Brain.h5ad',
    'image_path': '../tests/data/spatial/img.jpg',
    'annot_key': 'cluster',
}}
fs_cfg = {'SPARKX': [1000]}
cl_cfg = {'spaGCN': 1}

run_bench(data_cfg, fs_cfg, cl_cfg, ['ARI', 'NMI'], 'spatial', clean_cache=True)

2023-02-10 14:54:32.549 | INFO  | ./cache has been deleted.


─────────────────────────────────────────────────── mouse_brain ───────────────────────────────────────────────────

2023-02-10 14:54:32.554 | INFO  | Reading adata...
2023-02-10 14:54:32.555 | INFO  | No cache for mouse_brain. Trying to read h5ad from the given path in config...
2023-02-10 14:54:34.046 | INFO  | Using raw data in `adata.raw`...
2023-02-10 14:54:34.100 | INFO  | Found sparse matrix in `adata.X`. Converting to ndarray...


Before QC: 2688 cells and 18078 genes.

2023-02-10 14:54:34.931 | INFO  | Observation names and Variables names are all unique now.
2023-02-10 14:54:34.939 | INFO  | Dropping 0 special genes from 18078 genes...


After QC: 2688 cells and 16957 genes.

2023-02-10 14:54:39.665 | INFO  | Image has been loaded.


────────────────────────────────────── FS method: SPARKX, Gene Number: 1000 ───────────────────────────────────────

2023-02-10 14:54:39.668 | INFO  | Running SPARKX feature selection with 1000 features to be selected.
2023-02-10 14:55:16.758 | INFO  | Feature selection finished.
2023-02-10 14:55:16.763 | INFO  | Selected genes have been cached.
2023-02-10 14:55:17.027 | INFO  | Running spaGCN clustering with 15 clusters and random state 0...
2023-02-10 14:55:29.956 | INFO  | spaGCN clustering results have been cached.
2023-02-10 14:55:30.016 | INFO  | records have been saved into './2023-02 14_55_29 spatial.xlsx'.


## Evaluate custom methods defined by the user

In [6]:
data_cfg = {'mouse_brain': {
    'adata_path': '../tests/data/spatial/V1_Adult_Mouse_Brain.h5ad',
    'image_path': '../tests/data/spatial/img.jpg',
    'annot_key': 'cluster',
}}
fs_cfg = {random_select: [2000]}
cl_cfg = {'spaGCN': 1,  random_clustering: 1}

run_bench(data_cfg, fs_cfg, cl_cfg, ['ARI', 'NMI'], 'spatial', clean_cache=True, fs_kwarg={'seed': 123}, cl_kwarg={'k':5, 'seed': 123})

2023-02-10 14:55:30.107 | INFO  | ./cache has been deleted.


─────────────────────────────────────────────────── mouse_brain ───────────────────────────────────────────────────

2023-02-10 14:55:30.112 | INFO  | Reading adata...
2023-02-10 14:55:30.112 | INFO  | No cache for mouse_brain. Trying to read h5ad from the given path in config...
2023-02-10 14:55:31.594 | INFO  | Using raw data in `adata.raw`...
2023-02-10 14:55:31.650 | INFO  | Found sparse matrix in `adata.X`. Converting to ndarray...


Before QC: 2688 cells and 18078 genes.

2023-02-10 14:55:32.491 | INFO  | Observation names and Variables names are all unique now.
2023-02-10 14:55:32.502 | INFO  | Dropping 0 special genes from 18078 genes...


After QC: 2688 cells and 16957 genes.

2023-02-10 14:55:37.245 | INFO  | Image has been loaded.


─────────────────────────────────── FS method: random_select, Gene Number: 2000 ───────────────────────────────────

2023-02-10 14:55:37.252 | INFO  | Selected genes have been cached.
2023-02-10 14:55:37.574 | INFO  | Running spaGCN clustering with 15 clusters and random state 0...
2023-02-10 14:55:43.715 | INFO  | spaGCN clustering results have been cached.
2023-02-10 14:55:43.733 | INFO  | random_clustering clustering results have been cached.
2023-02-10 14:55:43.792 | INFO  | records have been saved into './2023-02 14_55_43 spatial.xlsx'.
